In [1]:
from astropy.table import Table, vstack, unique, SortedArray
import glob
import numpy
import time
from datetime import date, timedelta, datetime
import sqlite3
import pandas as pd
from functools import reduce
from desispec.io import read_spectra, write_spectra
from desispec.spectra import Spectra
from desispec.coaddition import coadd

from updateNoExpid import update_no_expid

import matplotlib.pyplot as plt
import os
from desiutil.log import get_logger, DEBUG

In [4]:
#get array of yyyymmdds to loop through
#getUnprocessedDates.py
start_time = time.time()

filename_trans = "/global/cfs/cdirs/desi/science/td/daily-search/transients_search.db"
filename_desi = "/global/cfs/cdirs/desi/science/td/db/desi.db"

con = sqlite3.connect(filename_desi)
con.execute("ATTACH ? AS ts", [filename_trans])

query_arr = con.execute("""SELECT DISTINCT yyyymmdd FROM main.fibermap_daily WHERE yyyymmdd NOT IN (SELECT yyyymmdd FROM ts.unprocessed_exposures)""").fetchall() #where to looking for this comparison now?
con.close()

print("--- query db:  %s seconds ---" % (time.time() - start_time))

--- 0.1430344581604004 seconds ---


In [ ]:
for a in query_arr:
    start_time = time.time()
    #get time duration stamp on process that reads in all spectra files for one night
    yyyymmdd = a
    # read in and store in one place all the fibermap information in the spectra files
    dats=[]
    for filename in glob.glob(f"/global/project/projectdirs/desi/spectro/redux/daily/tiles/cumulative/*/{yyyymmdd}/spectra-*.fits"):
        t = Table.read(filename, format='fits',hdu=1, memmap=True)[['TARGETID','TARGET_RA','TARGET_DEC','TILEID','OBJTYPE','PETAL_LOC','NIGHT','MJD']]
        t=t[t['OBJTYPE']=='TGT']
        dats.append(t)
    dats=vstack(dats, join_type='inner',metadata_conflicts='silent')
    # group all the observations by TARGET_RA and TARGET_DEC
    # note that this is more reliable than grouping by TARGETID as TARGETID is NOT a unique identifier of RA and DEC
    dats_group = dats.group_by(['TARGET_RA','TARGET_DEC'])
    print("--- read in file:  %s seconds ---" % (time.time() - start_time))
    #get RA_DEC, NIGHT for multiple pairs of tile/petal
    
    start_time = time.time()
    # for each group make a tuple containing all tileid/petal combinations that have that ra/dec
    ans=[]
    for g in dats_group.groups:
        #print(g)
        gu = unique(g,keys=['MJD']) 
        #print(gu)
        dum=[]
        for a,b,c,d,e in zip(gu['TARGET_RA'],gu['TARGET_DEC'],gu['NIGHT'],gu['TILEID'],gu['PETAL_LOC']):
            dum.append((a,b,c,d,e))
        ans.append(tuple(dum)) #can this still be a tuple?
    # compress things down to the unique tile/petal combinations
    ans = list(set(ans))
    print(len(ans))
    print("--- make list:  %s seconds ---" % (time.time() - start_time))
    
    start_time_i = time.time()
    for i in len(ans): #103753
        for j in len(ans[i]): #e.g. 3
            start_time_j = time.time()
            s0a = Spectra()
            print(s0a.num_spectra())
            while s0a.num_spectra()==0:
                for j in range(len(ans[i])):
                    if s0a.num_spectra()!=0: #this means a current spectra has been found and stored as s0a
                        if ans[i][j][2]==yyyymmdd:
                            f0b=f"/global/project/projectdirs/desi/spectro/redux/daily/tiles/cumulative/{ans[i][j][3]}/{ans[i][j][2]}/spectra-{ans[i][j][4]}-{ans[i][j][3]}-thru{ans[i][j][2]}.fits"
                            s0b=read_spectra(f0b)
                            print(s0b.num_spectra()) #1500
                            coadd(s0b)
                            print(s0b.num_spectra()) #500  
                            s0a.update_no_expid(s0b) #not working, produces NoneType
                            print(s0a.num_spectra()) #need to coadd?
                        elif ans[i][j][2] < yyyymmdd:
                            #makes reference spectra,r0a
                            f0a=f"/global/project/projectdirs/desi/spectro/redux/daily/tiles/cumulative/{ans[i][j][3]}/{ans[i][j][2]}/spectra-{ans[i][j][4]}-{ans[i][j][3]}-thru{ans[i][j][2]}.fits"
                            r0a=read_spectra(f0a)
                            print(r0a.num_spectra()) #1500
                            coadd(r0a)
                            print(r0a.num_spectra()) #500
                            #flip signs of r0a.flux
                            for i in r0a.flux:
                                r0a.flux[i] = -1*r0a.flux[i]
                            #flip signs of r0a.wave
                            for i in r0a.wave:
                                r0a.wave[i] = -1*r0a.wave[i]
                            d0a = Spectra()
                        d0a = Spectra()
                        d0a.update_no_expid(s0a)#not working, produce NoneType
                        d0a.update_no_expid(r0a)#not working, produces NoneType
                        coadd(d0a) #how to store differenced spectra as one per reference spectra will be produced
                    else if:
                        print(ans[i][j][2])
                        if ans[i][j][2]==yyyymmdd: #this makes a current spectra
                            f0a=f"/global/project/projectdirs/desi/spectro/redux/daily/tiles/cumulative/{ans[i][j][3]}/{ans[i][j][2]}/spectra-{ans[i][j][4]}-{ans[4][j][3]}-thru{ans[i][j][2]}.fits"
                            s0a=read_spectra(f0a)
                            print(s0a.num_spectra()) #1500
                            coadd(s0a)
                            print(s0a.num_spectra()) #500  
                        elif ans[i][j][2] < yyyymmdd:
                            #makes reference spectra,r0a
                            f0a=f"/global/project/projectdirs/desi/spectro/redux/daily/tiles/cumulative/{ans[i][j][3]}/{ans[i][j][2]}/spectra-{ans[i][j][4]}-{ans[i][j][3]}-thru{ans[i][j][2]}.fits"
                            r0a=read_spectra(f0a)
                            print(r0a.num_spectra()) #1500
                            coadd(r0a)
                            print(r0a.num_spectra()) #500
                            #flip signs of r0a.flux
                            for i in r0a.flux:
                                r0a.flux[i] = -1*r0a.flux[i]
                            #flip signs of r0a.wave
                            for i in r0a.wave:
                                r0a.wave[i] = -1*r0a.wave[i]
                        d0a = Spectra()
                        d0a.update_no_expid(s0a)#not working, produce NoneType
                        d0a.update_no_expid(r0a)#not working, produces NoneType
                        coadd(d0a)   #how to store differenced spectra as one per reference spectra will be produced
                        print("--- j:  %s seconds ---" % (time.time() - start_time))
    print("--- i:  %s seconds ---" % (time.time() - start_time))